In [8]:
from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D
from keras.layers import Activation, Dropout, Flatten, Dense
from keras import backend as K
from keras.preprocessing.image import ImageDataGenerator

In [41]:
train_data_dir = 'data/task1/train'
validation_data_dir = 'data/task1/validation'
nb_train_samples = 2000
nb_validation_samples = 800
epochs = 30
batch_size = 16
img_width, img_height = 150,150

In [42]:
def simple_model():
    model = Sequential()
    model.add(Conv2D(32, (3, 3), input_shape=(150, 150, 3)))
    model.add(Activation('relu'))
    model.add(MaxPooling2D(pool_size=(2, 2)))

    model.add(Conv2D(32, (3, 3)))
    model.add(Activation('relu'))
    model.add(MaxPooling2D(pool_size=(2, 2)))

    model.add(Conv2D(64, (3, 3)))
    model.add(Activation('relu'))
    model.add(MaxPooling2D(pool_size=(2, 2)))

    model.add(Flatten())  # this converts our 3D feature maps to 1D feature vectors
    model.add(Dense(64))
    model.add(Activation('relu'))
    model.add(Dropout(0.5))
    model.add(Dense(1))
    model.add(Activation('sigmoid'))

    model.compile(loss='binary_crossentropy',
                  optimizer='rmsprop',
                  metrics=['accuracy', precision, recall, f1_score])
    return model


In [43]:
model = simple_model()

In [51]:
# this is the augmentation configuration we will use for training
train_datagen = ImageDataGenerator(
    rescale=1. / 255,
    shear_range=0.1,
    horizontal_flip=True)

In [45]:

# this is the augmentation configuration we will use for testing:
# only rescaling
test_datagen = ImageDataGenerator(rescale=1. / 255)

In [46]:

train_generator = train_datagen.flow_from_directory(
    train_data_dir,
    target_size=(img_width, img_height),
    batch_size=batch_size,
    class_mode='binary')


Found 277 images belonging to 2 classes.


In [47]:

validation_generator = test_datagen.flow_from_directory(
    validation_data_dir,
    target_size=(img_width, img_height),
    batch_size=batch_size,
    class_mode='binary')

Found 32 images belonging to 2 classes.


In [48]:
def f1_score(true, pred):
    p = precision(true, pred)
    r = recall(true, pred)
    return 2 * (p * r) / (p + r + 1e-6)

def recall(y_true, y_pred):
    true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
    possible_positives = K.sum(K.round(K.clip(y_true, 0, 1)))
    recall = true_positives / (possible_positives + K.epsilon())
    return recall

def precision(y_true, y_pred):
    true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
    predicted_positives = K.sum(K.round(K.clip(y_pred, 0, 1)))
    precision = true_positives / (predicted_positives + K.epsilon())
    return precision

In [52]:
model.fit_generator(
    train_generator,
    steps_per_epoch=nb_train_samples // batch_size,
    epochs=epochs,
    validation_data=validation_generator,
    validation_steps=nb_validation_samples // batch_size)

Epoch 1/30
125/125 [==============================] - 38s 307ms/step - loss: 0.1413 - acc: 0.9633 - precision: 0.9729 - recall: 0.9829 - f1_score: 0.9764 - val_loss: 0.6648 - val_acc: 0.9062 - val_precision: 0.9228 - val_recall: 0.9594 - val_f1_score: 0.9395
Epoch 2/30
125/125 [==============================] - 39s 310ms/step - loss: 0.1031 - acc: 0.9637 - precision: 0.9758 - recall: 0.9807 - f1_score: 0.9768 - val_loss: 0.5512 - val_acc: 0.8750 - val_precision: 0.8623 - val_recall: 1.0000 - val_f1_score: 0.9246
Epoch 3/30
125/125 [==============================] - 39s 311ms/step - loss: 0.0899 - acc: 0.9693 - precision: 0.9791 - recall: 0.9833 - f1_score: 0.9799 - val_loss: 0.3477 - val_acc: 0.9062 - val_precision: 0.9250 - val_recall: 0.9592 - val_f1_score: 0.9404
Epoch 4/30
125/125 [==============================] - 41s 327ms/step - loss: 0.1176 - acc: 0.9655 - precision: 0.9731 - recall: 0.9845 - f1_score: 0.9778 - val_loss: 1.3910 - val_acc: 0.8750 - val_precision: 0.8617 - val_re

KeyboardInterrupt: 

In [ ]:
model.save_weights('third_try.h5')  # always save your weights after training or during training